OrganMNIST3D Basic Rundown:

What is OrganMNIST3D?

OrganMNIST3D, or OM3D for short, is a collection of CT scans of 11 various human organs such as the liver, spleen, pancreas. These scans are stored as 28x28x28 3D images sliced along the three major medical axies of axial, coronal, sagittal as grey scale pixel images.

What is the medical issue within OM3D?

Our medical issue is to make an AI that can correctly determine what human organ is being displayed in each CT scan with high accuracy. This data set is one of few that use 3D cross sections of humans to train AI models due to the effort getting these scan models and the ethical issues concerning the aquistion. 

In [1]:


%matplotlib qt
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # sometimes needed to register 3D
import numpy as np
from matplotlib import colors
from matplotlib.widgets import Slider




from medmnist import OrganMNIST3D

# Or NoduleMNIST3D, AdrenalMNIST3D, etc.
# Change these to match which dataset you've been assigned


train_dataset = OrganMNIST3D(split='train', size=28, download=True)
trainx = []
trainy = []

test_dataset = OrganMNIST3D(split='test', size=28, download=True)
testx = []
testy = []

val_dataset = OrganMNIST3D(split='train', size=28, download=True)
valx = []
valy = []

for i in range(len(train_dataset)):
    trainx.append(train_dataset[i][0])
    trainy.append(train_dataset[i][1])

for i in range(len(test_dataset)):
    testx.append(test_dataset[i][0])
    testy.append(test_dataset[i][1])

for i in range(len(val_dataset)):
    valx.append(val_dataset[i][0])
    valy.append(val_dataset[i][1])


trainx_tensor = tf.convert_to_tensor(trainx, dtype=tf.float16)
trainy_tensor = tf.convert_to_tensor(trainy, dtype=tf.float16)
testx_tensor = tf.convert_to_tensor(testx, dtype=tf.float16)
testy_tensor = tf.convert_to_tensor(testy, dtype=tf.float16)
valx_tensor = tf.convert_to_tensor(valx, dtype=tf.float16)
valy_tensor = tf.convert_to_tensor(valy, dtype=tf.float16)
# float16 doesn't run any faster on the 4090s, but it cuts memory usage in half!



In [7]:
def MyNet():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(1, 28, 28, 28), name='input'),        # was going to make it start with a dense layer, but too many issues with intillization when moving to Conv3D
        tf.keras.layers.Conv3D(filters=32, kernel_size=3, input_shape=(1, 28, 28, 28), activation='relu', padding='same', name='conv1', data_format='channels_first'),      
        tf.keras.layers.MaxPool3D(pool_size=2, data_format='channels_first'),
        tf.keras.layers.Conv3D(64, kernel_size=3, padding='same', activation='relu', data_format='channels_first'),
        tf.keras.layers.MaxPool3D(pool_size=2, data_format='channels_first'),
        tf.keras.layers.Conv3D(128, kernel_size=3, padding='same', activation='relu', data_format='channels_first'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu', name='dense1'),   # A 28 dense layer that smooths out Conv3D outputs
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(11, activation='softmax', name='output') # A final dense layer for the 11 organ classes

    ])
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [8]:

model = MyNet()
model.summary()
training_history = model.fit(trainx_tensor, trainy_tensor, epochs=5, validation_data=(valx_tensor, valy_tensor))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv3D)                  │ (None, 32, 28, 28, 28) │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 32, 14, 14, 14) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 64, 14, 14, 14) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 64, 7, 7, 7)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 128, 7, 7, 7)   │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 43904)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 256)            │    11,239,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,520,075 (43.95 MB)

 Trainable params: 11,520,075 (43.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.1627 - loss: 2.2770 - val_accuracy: 0.4789 - val_loss: 1.6759
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.5170 - loss: 1.3287 - val_accuracy: 0.8012 - val_loss: 0.8129
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.7199 - loss: 0.7708 - val_accuracy: 0.8558 - val_loss: 0.5075
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.8054 - loss: 0.5674 - val_accuracy: 0.8991 - val_loss: 0.3410
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.8558 - loss: 0.4434 - val_accuracy: 0.9310 - val_loss: 0.2442
